In [ ]:
from skyfield.api import Topos, load, EarthSatellite
from datetime import datetime
import requests


# Function to download TLE data from CelesTrak
def download_tle():
    url = 'https://celestrak.org/NORAD/elements/gp.php?GROUP=active&FORMAT=tle'
    response = requests.get(url)
    response.raise_for_status()  # Check if the request was successful
    tle_lines = response.text.strip().splitlines()
    return tle_lines

# Load satellites using EarthSatellite class
def load_satellites(tle_lines):
    satellites = []
    for i in range(0, len(tle_lines), 3):
        name = tle_lines[i].strip()
        if "STARLINK" not in name.upper():  # Exclude Starlink satellites
            line1 = tle_lines[i + 1].strip()
            line2 = tle_lines[i + 2].strip()
            satellite = EarthSatellite(line1, line2, name, load.timescale())
            satellites.append(satellite)
    return satellites

# Function to predict satellite culmination passes for a given location and time range
def predict_passes(satellites, location, start_time, end_time):
    ts = load.timescale()
    observer = Topos(latitude_degrees=location[0], longitude_degrees=location[1])

    print(f"Predicting satellite culminations for location: {location}")
    for satellite in satellites:
        t, events = satellite.find_events(observer, start_time, end_time, altitude_degrees=30.0)
        for ti, event in zip(t, events):
            if event == 1:  # 1 corresponds to 'culminate'
                # Calculate satellite altitude during culmination
                difference = satellite - observer
                topocentric = difference.at(ti)
                altitude = topocentric.distance().km  # Altitude in kilometers
                print(f"{satellite.name} culminate at {ti.utc_strftime('%Y-%m-%d %H:%M:%S')} UTC, altitude: {altitude:.2f} km")

# Main function
if __name__ == "__main__":
    # Download and load TLE data
    tle_lines = download_tle()
    satellites = load_satellites(tle_lines)

    # Set location for Zurich, Switzerland (latitude, longitude)
    location = (47.3769, 8.5417)

    # Set prediction time range (8 Jan 2025, 01:00 UTC to 8 Jan 2025, 02:00 UTC)
    ts = load.timescale()
    start_time = ts.utc(2025, 1, 7, 17, 15, 0)  # 8 Jan 2025, 01:00 UTC
    end_time = ts.utc(2025, 1, 7, 17, 40, 0)    # 8 Jan 2025, 02:00 UTC

    # Predict satellite culmination passes
    predict_passes(satellites, location, start_time, end_time)